In [1]:
!pip install pyspark


In [2]:
from google.colab import files

# Faça o upload do arquivo local
uploaded = files.upload()

Saving nomes_aleatorios.txt to nomes_aleatorios.txt


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder \
    .appName("Exercício Intro") \
    .master("local[*]") \
    .getOrCreate()

In [5]:
df_nomes = spark.read.csv('/content/nomes_aleatorios.txt', header=False, inferSchema=True)

df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [6]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

In [7]:
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [8]:
df_nomes = df_nomes.withColumn(
    "Escolaridade",
    expr("CASE WHEN rand() < 0.33 THEN 'Fundamental' " +
         "WHEN rand() < 0.66 THEN 'Medio' ELSE 'Superior' END")
)

In [9]:
paises = ["Argentina", "Brasil", "Chile", "Colômbia", "Equador", "Guiana",
          "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Bolívia", "Guiana Francesa"]

df_nomes = df_nomes.withColumn(
    "País",
    expr("CASE " +
         "WHEN rand() < 0.08 THEN 'Argentina' " +
         "WHEN rand() < 0.16 THEN 'Brasil' " +
         "WHEN rand() < 0.24 THEN 'Chile' " +
         "WHEN rand() < 0.32 THEN 'Colômbia' " +
         "WHEN rand() < 0.40 THEN 'Equador' " +
         "WHEN rand() < 0.48 THEN 'Guiana' " +
         "WHEN rand() < 0.56 THEN 'Paraguai' " +
         "WHEN rand() < 0.64 THEN 'Peru' " +
         "WHEN rand() < 0.72 THEN 'Suriname' " +
         "WHEN rand() < 0.80 THEN 'Uruguai' " +
         "WHEN rand() < 0.88 THEN 'Venezuela' " +
         "WHEN rand() < 0.96 THEN 'Bolívia' ELSE 'Guiana Francesa' END")
)

In [10]:
df_nomes = df_nomes.withColumn(
    "AnoNascimento",
    expr("CAST((1945 + rand() * (2010 - 1945)) AS INT)")
)

In [11]:
df_selec = df_nomes.filter(df_nomes["AnoNascimento"] >= 2000)
df_selec.show(10)

+----------------+------------+--------+-------------+
|           Nomes|Escolaridade|    País|AnoNascimento|
+----------------+------------+--------+-------------+
|   Jamie Russell|       Medio|  Brasil|         2005|
|  Edward Kistler|       Medio| Equador|         2001|
| Helen Blackwell|       Medio|Colômbia|         2003|
|Wallace Mitchell|    Superior|  Brasil|         2004|
|     Jessie Jean|    Superior|   Chile|         2004|
|     Lisa Baxley|    Superior|  Brasil|         2008|
|   Joann Ballard| Fundamental|  Brasil|         2000|
|    Curtis Sutch| Fundamental|Paraguai|         2004|
| Pamela Banuelos|       Medio|   Chile|         2001|
|     Shelia Ceja|       Medio| Bolívia|         2000|
+----------------+------------+--------+-------------+
only showing top 10 rows



In [12]:
df_nomes.createOrReplaceTempView("pessoas")
df_selec_sql = spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000")
df_selec_sql.show(10)

+----------------+------------+--------+-------------+
|           Nomes|Escolaridade|    País|AnoNascimento|
+----------------+------------+--------+-------------+
|   Jamie Russell|       Medio|  Brasil|         2005|
|  Edward Kistler|       Medio| Equador|         2001|
| Helen Blackwell|       Medio|Colômbia|         2003|
|Wallace Mitchell|    Superior|  Brasil|         2004|
|     Jessie Jean|    Superior|   Chile|         2004|
|     Lisa Baxley|    Superior|  Brasil|         2008|
|   Joann Ballard| Fundamental|  Brasil|         2000|
|    Curtis Sutch| Fundamental|Paraguai|         2004|
| Pamela Banuelos|       Medio|   Chile|         2001|
|     Shelia Ceja|       Medio| Bolívia|         2000|
+----------------+------------+--------+-------------+
only showing top 10 rows



In [13]:
millennials_count = df_nomes.filter(
    (df_nomes["AnoNascimento"] >= 1980) &
    (df_nomes["AnoNascimento"] <= 1994)
).count()

print(f"Número de Millennials: {millennials_count}")

Número de Millennials: 2306746


In [14]:
millennials_count_sql = spark.sql(
    "SELECT COUNT(*) AS Millennials FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994"
)
millennials_count_sql.show()

+-----------+
|Millennials|
+-----------+
|    2306746|
+-----------+



In [16]:
from pyspark.sql.functions import col

df_nomes = df_nomes.withColumnRenamed("País", "Pais").withColumnRenamed("Geração", "Geracao")

df_nomes.createOrReplaceTempView("pessoas")

geracoes = spark.sql(
    """
    SELECT Pais, Geracao, COUNT(*) AS Quantidade
    FROM pessoas
    GROUP BY Pais, Geracao
    ORDER BY Pais, Geracao, Quantidade
    """
)

geracoes.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    246291|
|Argentina|   Geração X|    184261|
|Argentina|   Geração Z|    185213|
|Argentina| Millennials|    184511|
|  Bolívia|Baby Boomers|       395|
|  Bolívia|   Geração X|       308|
|  Bolívia|   Geração Z|       298|
|  Bolívia| Millennials|       292|
|   Brasil|Baby Boomers|    453865|
|   Brasil|   Geração X|    339665|
|   Brasil|   Geração Z|    340020|
|   Brasil| Millennials|    339580|
|    Chile|Baby Boomers|    571452|
|    Chile|   Geração X|    428507|
|    Chile|   Geração Z|    426950|
|    Chile| Millennials|    427903|
| Colômbia|Baby Boomers|    578774|
| Colômbia|   Geração X|    434204|
| Colômbia|   Geração Z|    432605|
| Colômbia| Millennials|    433529|
+---------+------------+----------+
only showing top 20 rows

